In [1]:
from deepface import DeepFace

In [2]:
models = [
  "VGG-Face", 
  "Facenet", 
  "Facenet512", 
  "OpenFace", 
  "DeepFace", 
  "DeepID", 
  "ArcFace", 
  "Dlib", 
  "SFace",
]

embedding_objs = DeepFace.represent(img_path = "data_color/imgs/0001.png", 
      model_name = "Facenet512"
)


In [8]:
len(embedding_objs[0]['embedding'])

512

In [14]:
objs = DeepFace.analyze(img_path = "data_color/imgs/0001.png", 
        actions = ['age', 'gender', 'race', 'emotion'], silent=True
)
objs

[{'age': 34,
  'region': {'x': 58, 'y': 29, 'w': 385, 'h': 385},
  'gender': {'Woman': 0.000281260440715414, 'Man': 99.99971389770508},
  'dominant_gender': 'Man',
  'race': {'asian': 0.17462649725013954,
   'indian': 0.5998794812637258,
   'black': 0.020987287075898024,
   'white': 74.23427547420714,
   'middle eastern': 9.168500999718963,
   'latino hispanic': 15.801730203731669},
  'dominant_race': 'white',
  'emotion': {'angry': 0.30072988010942936,
   'disgust': 4.289007904390019e-06,
   'fear': 14.772641658782959,
   'happy': 1.8564298748970032,
   'sad': 80.10688424110413,
   'surprise': 0.011470907338662073,
   'neutral': 2.9518447816371918},
  'dominant_emotion': 'sad'}]